In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import warnings
from tqdm import tqdm
import os
from pathlib import Path
# Define the device
device = "cuda" if torch.cuda.is_available() else "mps" if  torch.backends.mps.is_available() else "cpu" # torch.has_mps or
print("Using device:", device)
if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
elif (device == 'mps'):
    print(f"Device name: <mps>")
else:
    print("NOTE: If you have a GPU, consider using it for training.")
    print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
    print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
device = torch.device(device)

Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


### esm2 2560 + MolFormer


#### bottle neck 128

no relu activation

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        # refined_enzy_embed = self.relu(refined_enzy_embed)
        # refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:28<00:00, 108.93batch/s, train_loss=tensor(0.1278, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.14607202670586533 ACC=  0.8007038340433413 bacc=  0.6823219346806877 precision=  0.6980703745743473 specificity=  0.9330649219929542 sensitivity=  0.43157894736842106 recall=  0.43157894736842106 MCC=  0.4349398620290454 AUC=  0.8286166220785987 f1=  0.5333911535125759
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1461); Accuracy (80.07)


Epoch 1: 100%|██████████| 3118/3118 [00:28<00:00, 109.46batch/s, train_loss=tensor(0.2128, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.13894567056696797 ACC=  0.8207075384330431 bacc=  0.7150403058476589 precision=  0.7423117709437964 specificity=  0.9388525415198792 sensitivity=  0.49122807017543857 recall=  0.49122807017543857 MCC=  0.4992784149570424 AUC=  0.8562714662852401 f1=  0.5912162162162162
Epoch: 1 / 500, ############## the best accuracy in val  80.0704 at Epoch: 0  ##############
Performance in Val: Loss: (0.1389); Accuracy (82.07)


Epoch 2: 100%|██████████| 3118/3118 [00:28<00:00, 108.85batch/s, train_loss=tensor(0.1728, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.07352860623518744 ACC=  0.9333209853676607 bacc=  0.9015915732966033 precision=  0.9055597867479056 specificity=  0.968797181680926 sensitivity=  0.8343859649122807 recall=  0.8343859649122807 MCC=  0.8251908549863225 AUC=  0.9538423436548089 f1=  0.8685171658144631
Epoch: 297 / 500, ############## the best accuracy in val  93.5729 at Epoch: 294  ##############
Performance in Val: Loss: (0.0735); Accuracy (93.33)


Epoch 298: 100%|██████████| 3118/3118 [00:29<00:00, 106.01batch/s, train_loss=tensor(0.0885, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.07282924036820523 ACC=  0.9305426930913132 bacc=  0.8965534747790462 precision=  0.9038461538461539 specificity=  0.9685455460493206 sensitivity=  0.8245614035087719 recall=  0.8245614035087719 MCC=  0.8176030137250262 AUC=  0.9538683901500102 f1=  0.8623853211009175
Epoch: 298 / 500, ############## the best accuracy in val  93.5729 at Epoch: 294  ##############
Performance in Val: Loss: (0.0728); Accuracy (93.05)


Epoch 299: 100%|██████████| 3118/3118 [00:29<00:00, 105.15batch/s, train_loss=tensor(0.0390, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.07332231732878901 ACC=  0.9312835710316726 bacc=  0.8997574585684139 precision=  0.8992424242424243 specificity=  0.966532460996477 sensitivity=  0.8329824561403508 recall=  0.8329824561403508 MCC=  0.8199438568355221 AUC=  0.955404073848436 f1=  0.8648451730418943
Epoch: 299 / 500, ############## the best accuracy in val  93.5729 at Epoch: 294  ##############
Performance in Val: Loss: (0.0733); Accuracy (93.13)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck128_batch_16_no_relu.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.08353295558089695 ACC=  0.9177433894230769 bacc=  0.8741009015544825 precision=  0.8929384965831435 specificity=  0.9664354213425832 sensitivity=  0.7817663817663818 recall=  0.7817663817663818 MCC=  0.7823833618397961 AUC=  0.9387963006561251 f1=  0.8336624639222239


(0.08353295558089695, 0.9177433894230769, 0.8741009015544825)

#### bottle neck 128

change loss function


In [ ]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        score = torch.nn.functional.sigmoid(cos_sim)
        loss = loss_fn(score, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (score > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            score = torch.nn.functional.sigmoid(cosine_sim)
            loss = loss_fn(score, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_loss_fn_CE.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:30<00:00, 103.37batch/s, train_loss=tensor(0.6226, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.6044418314504906 ACC=  0.6951287275421374 bacc=  0.6209585110234065 precision=  0.42838626053143225 specificity=  0.778057372924006 sensitivity=  0.463859649122807 recall=  0.463859649122807 MCC=  0.23601321214288373 AUC=  0.6751331020051388 f1=  0.4454177897574124
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.6044); Accuracy (69.51)


Epoch 1: 100%|██████████| 3118/3118 [00:31<00:00, 98.69batch/s, train_loss=tensor(0.5346, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)] 


loss_sum=  0.5925036540045541 ACC=  0.7145767734765698 bacc=  0.6566753194006657 precision=  0.4645909645909646 specificity=  0.7793155510820332 sensitivity=  0.5340350877192982 recall=  0.5340350877192982 MCC=  0.30043002403607666 AUC=  0.7118133658252324 f1=  0.4968984655566438
Epoch: 1 / 500, ############## the best accuracy in val  69.5129 at Epoch: 0  ##############
Performance in Val: Loss: (0.5925); Accuracy (71.46)


Epoch 2: 100%|██████████| 3118/3118 [00:30<00:00, 103.69batch/s, train_loss=tensor(0.5511, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.5716205345103021 ACC=  0.7414335988145953 bacc=  0.7822722256067951 precision=  0.5059256232120964 specificity=  0.6957725213890287 sensitivity=  0.8687719298245614 recall=  0.8687719298245614 MCC=  0.4998553508237778 AUC=  0.8770574523878898 f1=  0.6394628099173554
Epoch: 297 / 500, ############## the best accuracy in val  76.8290 at Epoch: 11  ##############
Performance in Val: Loss: (0.5716); Accuracy (74.14)


Epoch 298: 100%|██████████| 3118/3118 [00:32<00:00, 94.63batch/s, train_loss=tensor(0.5456, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5709467232227325 ACC=  0.7438414521207631 bacc=  0.7823324415719721 precision=  0.5086776859504132 specificity=  0.7008052340211374 sensitivity=  0.863859649122807 recall=  0.863859649122807 MCC=  0.5004598120731741 AUC=  0.8758960435815255 f1=  0.6403120936280884
Epoch: 298 / 500, ############## the best accuracy in val  76.8290 at Epoch: 11  ##############
Performance in Val: Loss: (0.5709); Accuracy (74.38)


Epoch 299: 100%|██████████| 3118/3118 [00:33<00:00, 94.30batch/s, train_loss=tensor(0.4738, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)] 


loss_sum=  0.5722848012955231 ACC=  0.7434710131505834 bacc=  0.7811805684316478 precision=  0.5082850041425021 specificity=  0.7013085052843483 sensitivity=  0.8610526315789474 recall=  0.8610526315789474 MCC=  0.49853524049291725 AUC=  0.8713593621698938 f1=  0.6392289658765304
Epoch: 299 / 500, ############## the best accuracy in val  76.8290 at Epoch: 11  ##############
Performance in Val: Loss: (0.5723); Accuracy (74.35)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_loss_fn_CE.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck128_batch_16_only_one_bottleneck_layer.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.5764824043457898 ACC=  0.7504507211538461 bacc=  0.7357213569415161 precision=  0.519756200084069 specificity=  0.7668843093246276 sensitivity=  0.7045584045584046 recall=  0.7045584045584046 MCC=  0.43345397862126067 AUC=  0.8043081649131434 f1=  0.5982099661344945


(0.5764824043457898, 0.7504507211538461, 0.7357213569415161)

#### bottle neck 128

only one neural network layer  No relu


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        # self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        # self.batch_norm_enzy = nn.BatchNorm1d(2560)
        # self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        # refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        # refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        # refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        # refined_enzy_embed = self.relu(refined_enzy_embed)
        # refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:22<00:00, 136.28batch/s, train_loss=tensor(0.2389, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.4116210244022883 ACC=  0.7377292091127987 bacc=  0.5108099135609532 precision=  0.5584415584415584 specificity=  0.9914443885254152 sensitivity=  0.030175438596491227 recall=  0.030175438596491227 MCC=  0.08036941048455676 AUC=  0.6504402299155034 f1=  0.057256990679094545
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.4116); Accuracy (73.77)


Epoch 1: 100%|██████████| 3118/3118 [00:21<00:00, 144.77batch/s, train_loss=tensor(0.1442, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.33457107356743965 ACC=  0.73624745323208 bacc=  0.5107036085432504 precision=  0.5053763440860215 specificity=  0.98842476094615 sensitivity=  0.03298245614035088 recall=  0.03298245614035088 MCC=  0.07251978028653913 AUC=  0.7179462117800792 f1=  0.0619235836627141
Epoch: 1 / 500, ############## the best accuracy in val  73.7729 at Epoch: 0  ##############
Performance in Val: Loss: (0.3346); Accuracy (73.62)


Epoch 2: 100%|██████████| 3118/3118 [00:21<00:00, 144.94batch/s, train_loss=tensor(0.1876, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  1.0918739510358437 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.7987759030187447 f1=  nan
Epoch: 160 / 500, ############## the best accuracy in val  73.7729 at Epoch: 0  ##############
Performance in Val: Loss: (1.0919); Accuracy (73.61)


Epoch 161: 100%|██████████| 3118/3118 [00:22<00:00, 135.74batch/s, train_loss=tensor(0.0233, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


loss_sum=  1.0764881890906384 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.7695361075058054 f1=  nan
Epoch: 161 / 500, ############## the best accuracy in val  73.7729 at Epoch: 0  ##############
Performance in Val: Loss: (1.0765); Accuracy (73.61)


Epoch 162:  25%|██▍       | 777/3118 [00:05<00:17, 133.49batch/s, train_loss=tensor(0.0414, device='cuda:0', grad_fn=<MseLossBackward0>)]


KeyboardInterrupt: 

In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck128_batch_16_only_one_bottleneck_layer.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

#### bottle neck 128

without both bachnorm and L2 normalization


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        # self.batch_norm_enzy = nn.BatchNorm1d(2560)
        # self.batch_norm_smiles = nn.BatchNorm1d(768)
        # self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        # refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        # refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        # refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        # refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            break
        break

Epoch 0:   0%|          | 0/3118 [00:00<?, ?batch/s]


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:25<00:00, 122.41batch/s, train_loss=tensor(0.1608, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.17716443371137924 ACC=  0.7601407668086683 bacc=  0.5620433696218402 precision=  0.7355072463768116 specificity=  0.9816305988928032 sensitivity=  0.1424561403508772 recall=  0.1424561403508772 MCC=  0.24833047365717692 AUC=  0.6820875162238762 f1=  0.23868312757201646
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1772); Accuracy (76.01)


Epoch 1: 100%|██████████| 3118/3118 [00:25<00:00, 120.36batch/s, train_loss=tensor(0.1839, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.15066619386025787 ACC=  0.7855158362659752 bacc=  0.6098884856832569 precision=  0.8248175182481752 specificity=  0.9818822345244087 sensitivity=  0.23789473684210527 recall=  0.23789473684210527 MCC=  0.36527455010154825 AUC=  0.7814182537370098 f1=  0.369281045751634
Epoch: 1 / 500, ############## the best accuracy in val  76.0141 at Epoch: 0  ##############
Performance in Val: Loss: (0.1507); Accuracy (78.55)


Epoch 2: 100%|██████████| 3118/3118 [00:25<00:00, 121.46batch/s, train_loss=tensor(0.1877, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.06073501216283498 ACC=  0.9305426930913132 bacc=  0.8994792466823829 precision=  0.8959276018099548 specificity=  0.96527428283845 sensitivity=  0.8336842105263158 recall=  0.8336842105263158 MCC=  0.8181192553375093 AUC=  0.9527194306854202 f1=  0.8636859323882226
Epoch: 297 / 500, ############## the best accuracy in val  93.3136 at Epoch: 138  ##############
Performance in Val: Loss: (0.0607); Accuracy (93.05)


Epoch 298: 100%|██████████| 3118/3118 [00:27<00:00, 114.17batch/s, train_loss=tensor(0.0084, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.060040767080551456 ACC=  0.9298018151509538 bacc=  0.8985258566648124 precision=  0.8944193061840121 specificity=  0.964771011575239 sensitivity=  0.832280701754386 recall=  0.832280701754386 MCC=  0.8161667465709124 AUC=  0.9536247891999754 f1=  0.8622319156670302
Epoch: 298 / 500, ############## the best accuracy in val  93.3136 at Epoch: 138  ##############
Performance in Val: Loss: (0.0600); Accuracy (92.98)


Epoch 299: 100%|██████████| 3118/3118 [00:27<00:00, 115.29batch/s, train_loss=tensor(0.0209, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.06029925743750291 ACC=  0.9298018151509538 bacc=  0.8987509160419922 precision=  0.8938253012048193 specificity=  0.9645193759436336 sensitivity=  0.8329824561403508 recall=  0.8329824561403508 MCC=  0.8162129185944368 AUC=  0.9535119504851711 f1=  0.8623320014529604
Epoch: 299 / 500, ############## the best accuracy in val  93.3136 at Epoch: 138  ##############
Performance in Val: Loss: (0.0603); Accuracy (92.98)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck128_batch_16_no_normaliztion_no_batchnorm.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.06934459753336761 ACC=  0.9192457932692307 bacc=  0.8812475911945408 precision=  0.8820207091308441 specificity=  0.9616404815343808 sensitivity=  0.8008547008547009 recall=  0.8008547008547009 MCC=  0.7873348292388141 AUC=  0.9446617819143835 f1=  0.8394803643422428


(0.06934459753336761, 0.9192457932692307, 0.8812475911945408)

#### bottle neck 128

only one neural network layer  change relu lcoation


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        # self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        # self.batch_norm_enzy = nn.BatchNorm1d(2560)
        # self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        # refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        # refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        # refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)


        refined_enzy_embed = self.enzy_refine_layer_2(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:21<00:00, 143.51batch/s, train_loss=tensor(0.0851, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


loss_sum=  0.85584017797511 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.5604766950087852 f1=  nan
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.8558); Accuracy (73.61)


Epoch 1: 100%|██████████| 3118/3118 [00:21<00:00, 142.80batch/s, train_loss=tensor(0.1473, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.7553388577329336 ACC=  0.7366178922022597 bacc=  0.5010526315789474 precision=  1.0 specificity=  1.0 sensitivity=  0.002105263157894737 recall=  0.002105263157894737 MCC=  0.0393759600809533 AUC=  0.6091117703670349 f1=  0.004201680672268907
Epoch: 1 / 500, ############## the best accuracy in val  73.6062 at Epoch: 0  ##############
Performance in Val: Loss: (0.7553); Accuracy (73.66)


Epoch 2: 100%|██████████| 3118/3118 [00:22<00:00, 140.35batch/s, train_loss=tensor(0.1544, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


...

loss_sum=  0.19866770358767205 ACC=  0.7477310613076495 bacc=  0.5230055006666137 precision=  0.9436619718309859 specificity=  0.9989934574735783 sensitivity=  0.047017543859649125 recall=  0.047017543859649125 MCC=  0.17802121713735553 AUC=  0.8932981043449086 f1=  0.089572192513369
Epoch: 297 / 500, ############## the best accuracy in val  75.8289 at Epoch: 275  ##############
Performance in Val: Loss: (0.1987); Accuracy (74.77)


Epoch 298: 100%|██████████| 3118/3118 [00:23<00:00, 133.78batch/s, train_loss=tensor(0.0523, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.2177873576864512 ACC=  0.7475458418225598 bacc=  0.5224295640964515 precision=  0.9558823529411765 specificity=  0.9992450931051837 sensitivity=  0.0456140350877193 recall=  0.0456140350877193 MCC=  0.1773019022226592 AUC=  0.8661278132422148 f1=  0.08707300736771602
Epoch: 298 / 500, ############## the best accuracy in val  75.8289 at Epoch: 275  ##############
Performance in Val: Loss: (0.2178); Accuracy (74.75)


Epoch 299: 100%|██████████| 3118/3118 [00:23<00:00, 132.41batch/s, train_loss=tensor(0.0116, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.18900245479060082 ACC=  0.7497684756436377 bacc=  0.5264150310350612 precision=  0.9743589743589743 specificity=  0.9994967287367891 sensitivity=  0.05333333333333334 recall=  0.05333333333333334 MCC=  0.19514550569185538 AUC=  0.8969112388419465 f1=  0.10113107119095144
Epoch: 299 / 500, ############## the best accuracy in val  75.8289 at Epoch: 275  ##############
Performance in Val: Loss: (0.1890); Accuracy (74.98)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck128_batch_16_only_one_bottleneck_layer.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.18801534217625607 ACC=  0.7545072115384616 bacc=  0.5373075789521413 precision=  0.8980263157894737 specificity=  0.9968373801265048 sensitivity=  0.07777777777777778 recall=  0.07777777777777778 MCC=  0.22008777190733494 AUC=  0.8805120444632789 f1=  0.14315679077084426


(0.18801534217625607, 0.7545072115384616, 0.5373075789521413)

#### bottle neck 128

only one neural network layer

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        # self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        # self.batch_norm_enzy = nn.BatchNorm1d(2560)
        # self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        # refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        # refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        # refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(enzy_embed)
        refined_smiles_embed = self.relu(smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:22<00:00, 137.15batch/s, train_loss=tensor(0.1080, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


loss_sum=  0.4297755750656657 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.6412874032085751 f1=  nan
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.4298); Accuracy (73.61)


Epoch 1: 100%|██████████| 3118/3118 [00:21<00:00, 144.87batch/s, train_loss=tensor(0.0640, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


loss_sum=  0.4101071206841772 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.7249600473251574 f1=  nan
Epoch: 1 / 500, ############## the best accuracy in val  73.6062 at Epoch: 0  ##############
Performance in Val: Loss: (0.4101); Accuracy (73.61)


Epoch 2: 100%|██████████| 3118/3118 [00:21<00:00, 145.72batch/s, train_loss=tensor(0.0701, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


...

Epoch 298: 100%|██████████| 3118/3118 [00:22<00:00, 135.58batch/s, train_loss=tensor(0.0457, device='cuda:0', grad_fn=<MseLossBackward0>)]
<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


loss_sum=  1.0927101861266695 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.8254874226330801 f1=  nan
Epoch: 298 / 500, ############## the best accuracy in val  73.6062 at Epoch: 0  ##############
Performance in Val: Loss: (1.0927); Accuracy (73.61)


Epoch 299: 100%|██████████| 3118/3118 [00:22<00:00, 135.60batch/s, train_loss=tensor(0.0277, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  1.0904266274363332 ACC=  0.7360622337469902 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.8416419887161286 f1=  nan
Epoch: 299 / 500, ############## the best accuracy in val  73.6062 at Epoch: 0  ##############
Performance in Val: Loss: (1.0904); Accuracy (73.61)


<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


In [ ]:
loss

tensor(0.0277, device='cuda:0', grad_fn=<MseLossBackward0>)

In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck128_batch_16_only_one_bottleneck_layer.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.4331808256527158 ACC=  0.736328125 bacc=  0.5 precision=  nan specificity=  1.0 sensitivity=  0.0 recall=  0.0 MCC=  nan AUC=  0.6394697343585326 f1=  nan


<ipython-input-3-c7858174dfc2>:33: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
<ipython-input-3-c7858174dfc2>:35: RuntimeWarning: invalid value encountered in scalar divide
  MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


(0.4331808256527158, 0.736328125, 0.5)

#### bottle neck size 64

batch size 16

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 64) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 64) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(64)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:28<00:00, 111.12batch/s, train_loss=tensor(0.1115, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.15124919422566008 ACC=  0.7969994443415447 bacc=  0.6424457217527967 precision=  0.789103690685413 specificity=  0.9698037242073477 sensitivity=  0.31508771929824564 recall=  0.31508771929824564 MCC=  0.40895080811350093 AUC=  0.8183105095400808 f1=  0.45035105315947843
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1512); Accuracy (79.70)


Epoch 1: 100%|██████████| 3118/3118 [00:28<00:00, 111.27batch/s, train_loss=tensor(0.3087, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.13001846119254476 ACC=  0.8345989998147805 bacc=  0.7215508701295261 precision=  0.815914489311164 specificity=  0.9609964771011575 sensitivity=  0.48210526315789476 recall=  0.48210526315789476 MCC=  0.5383060479681975 AUC=  0.8641410395641846 f1=  0.6060873400970447
Epoch: 1 / 500, ############## the best accuracy in val  79.6999 at Epoch: 0  ##############
Performance in Val: Loss: (0.1300); Accuracy (83.46)


Epoch 2: 100%|██████████| 3118/3118 [00:28<00:00, 110.54batch/s, train_loss=tensor(0.1710, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.056422942477835 ACC=  0.9385071309501759 bacc=  0.9147920253578082 precision=  0.8986141502552881 specificity=  0.9650226472068445 sensitivity=  0.864561403508772 recall=  0.864561403508772 MCC=  0.8400754612861473 AUC=  0.9626783743455266 f1=  0.8812589413447782
Epoch: 297 / 500, ############## the best accuracy in val  94.0730 at Epoch: 198  ##############
Performance in Val: Loss: (0.0564); Accuracy (93.85)


Epoch 298: 100%|██████████| 3118/3118 [00:29<00:00, 107.32batch/s, train_loss=tensor(0.0098, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.05810676710735235 ACC=  0.9403593258010743 bacc=  0.9144747878755772 precision=  0.9094283593170007 specificity=  0.9693004529441369 sensitivity=  0.8596491228070176 recall=  0.8596491228070176 MCC=  0.8443664348706567 AUC=  0.9600163342427533 f1=  0.8838383838383839
Epoch: 298 / 500, ############## the best accuracy in val  94.0730 at Epoch: 198  ##############
Performance in Val: Loss: (0.0581); Accuracy (94.04)


Epoch 299: 100%|██████████| 3118/3118 [00:29<00:00, 105.80batch/s, train_loss=tensor(0.0177, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.056277966033116245 ACC=  0.9383219114650861 bacc=  0.9124156137702082 precision=  0.9038461538461539 specificity=  0.9672873678912934 sensitivity=  0.8575438596491228 recall=  0.8575438596491228 MCC=  0.839134371096149 AUC=  0.9637891028527534 f1=  0.8800864241987756
Epoch: 299 / 500, ############## the best accuracy in val  94.0730 at Epoch: 198  ##############
Performance in Val: Loss: (0.0563); Accuracy (93.83)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck64_batch_16.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.06297317462303909 ACC=  0.9344951923076923 bacc=  0.9055015227429021 precision=  0.9011557177615572 specificity=  0.96684350132626 sensitivity=  0.8441595441595442 recall=  0.8441595441595442 MCC=  0.8286038731147382 AUC=  0.9571352087573267 f1=  0.8717269785230951


(0.06297317462303909, 0.9344951923076923, 0.9055015227429021)

#### bottle neck 128


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.06416148592129153 ACC=  0.9347205528846154 bacc=  0.9052887921588186 precision=  0.9029600244125725 specificity=  0.9675576412976944 sensitivity=  0.843019943019943 recall=  0.843019943019943 MCC=  0.8291031932778974 AUC=  0.956207422637743 f1=  0.8719611021069692


(0.06416148592129153, 0.9347205528846154, 0.9052887921588186)

#### large bottle neck size 256


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 256) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 256) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(256)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:28<00:00, 109.95batch/s, train_loss=tensor(0.2387, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.15676932122844917 ACC=  0.7888497869975921 bacc=  0.6434364597956895 precision=  0.7123695976154992 specificity=  0.951434323100151 sensitivity=  0.3354385964912281 recall=  0.3354385964912281 MCC=  0.38327538001788575 AUC=  0.7919251450215876 f1=  0.4561068702290076
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1568); Accuracy (78.88)


Epoch 1: 100%|██████████| 3118/3118 [00:28<00:00, 110.74batch/s, train_loss=tensor(0.0287, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.13401408999408843 ACC=  0.8210779774032229 bacc=  0.6819831536566631 precision=  0.8558139534883721 specificity=  0.9765978862606945 sensitivity=  0.3873684210526316 recall=  0.3873684210526316 MCC=  0.4946216251209706 AUC=  0.8669801958343266 f1=  0.5333333333333333
Epoch: 1 / 500, ############## the best accuracy in val  78.8850 at Epoch: 0  ##############
Performance in Val: Loss: (0.1340); Accuracy (82.11)


Epoch 2: 100%|██████████| 3118/3118 [00:27<00:00, 111.91batch/s, train_loss=tensor(0.1236, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.05690815052728461 ACC=  0.9392480088905353 bacc=  0.9150702372438393 precision=  0.9018315018315018 specificity=  0.9662808253648717 sensitivity=  0.863859649122807 recall=  0.863859649122807 MCC=  0.8418576832444375 AUC=  0.9608831969203332 f1=  0.882437275985663
Epoch: 297 / 500, ############## the best accuracy in val  94.2397 at Epoch: 155  ##############
Performance in Val: Loss: (0.0569); Accuracy (93.92)


Epoch 298: 100%|██████████| 3118/3118 [00:28<00:00, 107.77batch/s, train_loss=tensor(0.0466, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.05644252625591612 ACC=  0.9403593258010743 bacc=  0.914699847252757 precision=  0.9088213491475167 specificity=  0.9690488173125315 sensitivity=  0.8603508771929824 recall=  0.8603508771929824 MCC=  0.8444068508638799 AUC=  0.9609312284233482 f1=  0.8839221341023793
Epoch: 298 / 500, ############## the best accuracy in val  94.2397 at Epoch: 155  ##############
Performance in Val: Loss: (0.0564); Accuracy (94.04)


Epoch 299: 100%|██████████| 3118/3118 [00:29<00:00, 105.87batch/s, train_loss=tensor(0.0167, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.05645986904111862 ACC=  0.9401741063159844 bacc=  0.9145740294369542 precision=  0.9081481481481481 specificity=  0.968797181680926 sensitivity=  0.8603508771929824 recall=  0.8603508771929824 MCC=  0.843942152646503 AUC=  0.9598372756248951 f1=  0.8836036036036036
Epoch: 299 / 500, ############## the best accuracy in val  94.2397 at Epoch: 155  ##############
Performance in Val: Loss: (0.0565); Accuracy (94.02)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck256_batch_size_16.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.06434282093519308 ACC=  0.9317908653846154 bacc=  0.9014705993485834 precision=  0.8971306471306472 specificity=  0.9656192613752296 sensitivity=  0.8373219373219373 recall=  0.8373219373219373 MCC=  0.8213766951188213 AUC=  0.9566201240400384 f1=  0.8661951075744179


(0.06434282093519308, 0.9317908653846154, 0.9014705993485834)

#### large bottle neck size 512


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 512) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 512) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(512)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
1559


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 1559/1559 [00:14<00:00, 105.45batch/s, train_loss=tensor(0.0843, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.15330995596151734 ACC=  0.8081126134469346 bacc=  0.6752014409450905 precision=  0.7653478854024557 specificity=  0.9567186713638651 sensitivity=  0.3936842105263158 recall=  0.3936842105263158 MCC=  0.4508838485314203 AUC=  0.8290738042892839 f1=  0.519925857275255
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1533); Accuracy (80.81)


Epoch 1: 100%|██████████| 1559/1559 [00:14<00:00, 105.11batch/s, train_loss=tensor(0.1403, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.1268719949886291 ACC=  0.8303389516577144 bacc=  0.7139308134452891 precision=  0.8092345078979344 specificity=  0.9604932058379466 sensitivity=  0.4673684210526316 recall=  0.4673684210526316 MCC=  0.5246646164730729 AUC=  0.8823306757078908 f1=  0.5925266903914591
Epoch: 1 / 500, ############## the best accuracy in val  80.8113 at Epoch: 0  ##############
Performance in Val: Loss: (0.1269); Accuracy (83.03)


Epoch 2: 100%|██████████| 1559/1559 [00:14<00:00, 105.62batch/s, train_loss=tensor(0.1306, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.05704977013964272 ACC=  0.9381366919799963 bacc=  0.9136401522174837 precision=  0.8996336996336997 specificity=  0.9655259184700553 sensitivity=  0.8617543859649123 recall=  0.8617543859649123 MCC=  0.8389571426633432 AUC=  0.9602492517150956 f1=  0.8802867383512545
Epoch: 297 / 500, ############## the best accuracy in val  93.9804 at Epoch: 259  ##############
Performance in Val: Loss: (0.0570); Accuracy (93.81)


Epoch 298: 100%|██████████| 1559/1559 [00:15<00:00, 101.26batch/s, train_loss=tensor(0.0131, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.057188510922329315 ACC=  0.9372105945545471 bacc=  0.9139113006471893 precision=  0.8940493468795355 specificity=  0.9632611977856065 sensitivity=  0.864561403508772 recall=  0.864561403508772 MCC=  0.8368873389863701 AUC=  0.9610825629751278 f1=  0.8790581519800214
Epoch: 298 / 500, ############## the best accuracy in val  93.9804 at Epoch: 259  ##############
Performance in Val: Loss: (0.0572); Accuracy (93.72)


Epoch 299: 100%|██████████| 1559/1559 [00:15<00:00, 101.82batch/s, train_loss=tensor(0.0132, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.057085281474923595 ACC=  0.9368401555843675 bacc=  0.913434605638404 precision=  0.8933236574746009 specificity=  0.963009562154001 sensitivity=  0.863859649122807 recall=  0.863859649122807 MCC=  0.8359235092557333 AUC=  0.962047960868452 f1=  0.8783446307527649
Epoch: 299 / 500, ############## the best accuracy in val  93.9804 at Epoch: 259  ##############
Performance in Val: Loss: (0.0571); Accuracy (93.68)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_botlle_neck512_batch_size_16.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.0632145201223186 ACC=  0.9313401442307693 bacc=  0.9036334232620704 precision=  0.8890887290167866 specificity=  0.9622526015098959 sensitivity=  0.845014245014245 recall=  0.845014245014245 MCC=  0.8207989326152744 AUC=  0.9566620655939163 f1=  0.8664913818288051


(0.0632145201223186, 0.9313401442307693, 0.9036334232620704)

#### no batchnorm layer bottle_neck_512

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 512) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 512) # W1 and b

        self.relu = nn.ReLU()
        # self.batch_norm_enzy = nn.BatchNorm1d(2560)
        # self.batch_norm_smiles = nn.BatchNorm1d(768)
        # self.batch_norm_shared = nn.BatchNorm1d(512)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        # refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        # refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:27<00:00, 114.48batch/s, train_loss=tensor(0.2001, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.16308579123275874 ACC=  0.7786627153176514 bacc=  0.5957807326570074 precision=  0.8159340659340659 specificity=  0.9831404126824358 sensitivity=  0.20842105263157895 recall=  0.20842105263157895 MCC=  0.3367278867041013 AUC=  0.719949849460087 f1=  0.33202906651760766
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1631); Accuracy (77.87)


Epoch 1: 100%|██████████| 3118/3118 [00:27<00:00, 114.52batch/s, train_loss=tensor(0.1930, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.14659759460138322 ACC=  0.7882941285423226 bacc=  0.6151516435779938 precision=  0.8309859154929577 specificity=  0.9818822345244087 sensitivity=  0.24842105263157896 recall=  0.24842105263157896 MCC=  0.3765368379088788 AUC=  0.7940666966863561 f1=  0.3824959481361427
Epoch: 1 / 500, ############## the best accuracy in val  77.8663 at Epoch: 0  ##############
Performance in Val: Loss: (0.1466); Accuracy (78.83)


Epoch 2: 100%|██████████| 3118/3118 [00:26<00:00, 116.99batch/s, train_loss=tensor(0.1016, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.05892670938687278 ACC=  0.9327653269123912 bacc=  0.9000888229632965 precision=  0.9065849923430321 specificity=  0.9693004529441369 sensitivity=  0.8308771929824561 recall=  0.8308771929824561 MCC=  0.8235980645540549 AUC=  0.9574408214799707 f1=  0.8670816550714024
Epoch: 297 / 500, ############## the best accuracy in val  93.7581 at Epoch: 111  ##############
Performance in Val: Loss: (0.0589); Accuracy (93.28)


Epoch 298: 100%|██████████| 3118/3118 [00:27<00:00, 111.93batch/s, train_loss=tensor(0.0076, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.05915959175406079 ACC=  0.9323948879422115 bacc=  0.8996121279545113 precision=  0.9058192955589587 specificity=  0.9690488173125315 sensitivity=  0.8301754385964912 recall=  0.8301754385964912 MCC=  0.8226167697412926 AUC=  0.9568798064612967 f1=  0.8663493225924569
Epoch: 298 / 500, ############## the best accuracy in val  93.7581 at Epoch: 111  ##############
Performance in Val: Loss: (0.0592); Accuracy (93.24)


Epoch 299: 100%|██████████| 3118/3118 [00:28<00:00, 110.67batch/s, train_loss=tensor(0.0154, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.058728609617782986 ACC=  0.9349879607334691 bacc=  0.9027239336388277 precision=  0.9118098159509203 specificity=  0.9710619023653749 sensitivity=  0.8343859649122807 recall=  0.8343859649122807 MCC=  0.8294554225953766 AUC=  0.9571160790753935 f1=  0.871381458409674
Epoch: 299 / 500, ############## the best accuracy in val  93.7581 at Epoch: 111  ##############
Performance in Val: Loss: (0.0587); Accuracy (93.50)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_no_BatchNorm_batch_16.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.0698143583975672 ACC=  0.9213491586538461 bacc=  0.8845046740272204 precision=  0.8849640512660206 specificity=  0.9624566415017344 sensitivity=  0.8065527065527065 recall=  0.8065527065527065 MCC=  0.7930405653295376 AUC=  0.945486923129241 f1=  0.8439409748099567


(0.0698143583975672, 0.9213491586538461, 0.8845046740272204)

#### no normalization layer


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        # refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        # refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:27<00:00, 113.44batch/s, train_loss=tensor(0.2370, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.14120339387311384 ACC=  0.8084830524171143 bacc=  0.6761282547082351 precision=  0.7652645861601085 specificity=  0.9564670357322597 sensitivity=  0.3957894736842105 recall=  0.3957894736842105 MCC=  0.45223119035631654 AUC=  0.8322684289990199 f1=  0.5217391304347826
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1412); Accuracy (80.85)


Epoch 1: 100%|██████████| 3118/3118 [00:26<00:00, 119.12batch/s, train_loss=tensor(0.0918, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.13564665356551756 ACC=  0.8229301722541211 bacc=  0.693819122542138 precision=  0.821673525377229 specificity=  0.9672873678912934 sensitivity=  0.4203508771929825 recall=  0.4203508771929825 MCC=  0.4999490850056226 AUC=  0.8517324892503025 f1=  0.5561745589600743
Epoch: 1 / 500, ############## the best accuracy in val  80.8483 at Epoch: 0  ##############
Performance in Val: Loss: (0.1356); Accuracy (82.29)


Epoch 2: 100%|██████████| 3118/3118 [00:26<00:00, 115.76batch/s, train_loss=tensor(0.0934, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.05660757425478136 ACC=  0.9398036673458048 bacc=  0.9147725125597083 precision=  0.9056047197640118 specificity=  0.9677906391545043 sensitivity=  0.8617543859649123 recall=  0.8617543859649123 MCC=  0.843101013319121 AUC=  0.9628132863613488 f1=  0.8831355627472132
Epoch: 297 / 500, ############## the best accuracy in val  94.1841 at Epoch: 262  ##############
Performance in Val: Loss: (0.0566); Accuracy (93.98)


Epoch 298: 100%|██████████| 3118/3118 [00:27<00:00, 112.95batch/s, train_loss=tensor(0.0398, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.0560620933986054 ACC=  0.9396184478607149 bacc=  0.9146466947439056 precision=  0.9049373618275608 specificity=  0.9675390035228988 sensitivity=  0.8617543859649123 recall=  0.8617543859649123 MCC=  0.8426388697760924 AUC=  0.9621916139114771 f1=  0.8828181164629763
Epoch: 298 / 500, ############## the best accuracy in val  94.1841 at Epoch: 262  ##############
Performance in Val: Loss: (0.0561); Accuracy (93.96)


Epoch 299: 100%|██████████| 3118/3118 [00:27<00:00, 113.49batch/s, train_loss=tensor(0.0239, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.056380588109412134 ACC=  0.9411002037414335 bacc=  0.9156532372703273 precision=  0.910303928836175 specificity=  0.9695520885757424 sensitivity=  0.8617543859649123 recall=  0.8617543859649123 MCC=  0.846348132173036 AUC=  0.9624064312769847 f1=  0.8853640951694304
Epoch: 299 / 500, ############## the best accuracy in val  94.1841 at Epoch: 262  ##############
Performance in Val: Loss: (0.0564); Accuracy (94.11)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_no_normalize_batch_16.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.062299701555205575 ACC=  0.9326923076923077 bacc=  0.9011683178791932 precision=  0.9028976572133169 specificity=  0.967863701285452 sensitivity=  0.8344729344729345 recall=  0.8344729344729345 MCC=  0.8234846424730509 AUC=  0.9566515148080135 f1=  0.8673378738525318


(0.062299701555205575, 0.9326923076923077, 0.9011683178791932)

#### no batchnorm layer bottle_neck_128

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        # self.batch_norm_enzy = nn.BatchNorm1d(2560)
        # self.batch_norm_smiles = nn.BatchNorm1d(768)
        # self.batch_norm_shared = nn.BatchNorm1d(512)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        # refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        # refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        # refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3118/3118 [00:25<00:00, 122.56batch/s, train_loss=tensor(0.2007, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.16454348785350836 ACC=  0.7708834969438785 bacc=  0.5911715625248324 precision=  0.7281553398058253 specificity=  0.9718168092601912 sensitivity=  0.21052631578947367 recall=  0.21052631578947367 MCC=  0.3027209412158319 AUC=  0.7155864876080489 f1=  0.326619488296135
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1645); Accuracy (77.09)


Epoch 1: 100%|██████████| 3118/3118 [00:25<00:00, 120.44batch/s, train_loss=tensor(0.1428, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.14542806977351036 ACC=  0.7932950546397481 bacc=  0.637678771664945 precision=  0.7715289982425307 specificity=  0.9672873678912934 sensitivity=  0.3080701754385965 recall=  0.3080701754385965 MCC=  0.39526525780931693 AUC=  0.8005553642536133 f1=  0.440320962888666
Epoch: 1 / 500, ############## the best accuracy in val  77.0883 at Epoch: 0  ##############
Performance in Val: Loss: (0.1454); Accuracy (79.33)


Epoch 2: 100%|██████████| 3118/3118 [00:25<00:00, 120.16batch/s, train_loss=tensor(0.0847, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.061802145818519796 ACC=  0.9303574736062233 bacc=  0.9004787257524789 precision=  0.8922961854899027 specificity=  0.9637644690488173 sensitivity=  0.8371929824561404 recall=  0.8371929824561404 MCC=  0.8178904674170626 AUC=  0.9486533520514925 f1=  0.8638667632150616
Epoch: 297 / 500, ############## the best accuracy in val  93.5358 at Epoch: 224  ##############
Performance in Val: Loss: (0.0618); Accuracy (93.04)


Epoch 298: 100%|██████████| 3118/3118 [00:27<00:00, 112.63batch/s, train_loss=tensor(0.0107, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.060583786633535225 ACC=  0.9338766438229301 bacc=  0.9039945611386292 precision=  0.9021084337349398 specificity=  0.9672873678912934 sensitivity=  0.840701754385965 recall=  0.840701754385965 MCC=  0.8269462628858608 AUC=  0.9508582099435806 f1=  0.8703232836905195
Epoch: 298 / 500, ############## the best accuracy in val  93.5358 at Epoch: 224  ##############
Performance in Val: Loss: (0.0606); Accuracy (93.39)


Epoch 299: 100%|██████████| 3118/3118 [00:27<00:00, 112.95batch/s, train_loss=tensor(0.0084, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.06085357854498231 ACC=  0.9307279125764031 bacc=  0.900955420761264 precision=  0.8930441286462228 specificity=  0.9640161046804228 sensitivity=  0.8378947368421052 recall=  0.8378947368421052 MCC=  0.818864013022558 AUC=  0.9508031149842399 f1=  0.8645908761766834
Epoch: 299 / 500, ############## the best accuracy in val  93.5358 at Epoch: 224  ##############
Performance in Val: Loss: (0.0609); Accuracy (93.07)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_no_BatchNorm_batch_16_bottle128.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.07090326010704583 ACC=  0.9186448317307693 bacc=  0.880839511210864 precision=  0.8798122065727699 specificity=  0.9608243215670271 sensitivity=  0.8008547008547009 recall=  0.8008547008547009 MCC=  0.7858173183255264 AUC=  0.9414271376677008 f1=  0.8384787472035794


(0.07090326010704583, 0.9186448317307693, 0.880839511210864)